### Data import

In [ ]:
import kagglehub
path = kagglehub.dataset_download("csmalarkodi/isot-fake-news-dataset")
print("Path to dataset files:", path)
import shutil
shutil.copytree(path, 'data/')

In [87]:
import pandas as pd

## Data preparation

### Concatenation

In [4]:
df_true = pd.read_csv("data/True.csv")
df_true["fake"] = 0
df_true.head()

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [5]:
df_fake = pd.read_csv("data/Fake.csv")
df_fake["fake"] = 1
df_fake.head()

,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [28]:
df = pd.concat([df_true, df_fake], ignore_index=True)
df.head()

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


### Removing empty values.

In [29]:
df["text"].value_counts()

text
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [30]:
df = df[~df["text"].str.strip().eq("")]

### Removing duplicates.

In [31]:
df = df.drop_duplicates()
df.head()

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [35]:
df = df.reset_index(drop=True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44058 entries, 0 to 44057
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44058 non-null  object
 1   text     44058 non-null  object
 2   subject  44058 non-null  object
 3   date     44058 non-null  object
 4   fake     44058 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


### Removing wrong data.

In [44]:
df["date"] = df["date"].str.strip()

In [69]:
df = df[~df["date"].str.contains("http")]

In [78]:
df = df[~df["date"].str.contains("Video")]

In [ ]:
# df[df["text"].str.startswith("http")]["text"].apply(lambda x: " ".join(x.split()[1:]))

In [94]:
df.shape

(44048, 5)

In [96]:
df = df[~df["text"].str.startswith("http")]

In [99]:
df = df.reset_index(drop=True)

### Fixing dtype for date

In [106]:
df["date"] = pd.to_datetime(df["date"], format="mixed")

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43963 entries, 0 to 43962
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    43963 non-null  object        
 1   text     43963 non-null  object        
 2   subject  43963 non-null  object        
 3   date     43963 non-null  datetime64[ns]
 4   fake     43963 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.7+ MB


## EDA

## Clearing data for modeling

In [108]:
df_modeling = df.drop(["date", "subject"], axis=1)
df_modeling.head()

,title,text,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,0
